In [ ]:
from aquacrop.classes import *
from aquacrop.core import *
 
# import gym
# from gym import spaces
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
from aquacrop.lars import *
# import math

In [ ]:
from aquacropgym.utils import calc_eto_faopm

In [ ]:
#CP  Lat.(deg)= 40.40  Long.(deg)=101.73  Elev.(m)=1072.
gendf=calc_eto_faopm('data/CPWG.dat',2018,40.4,1072,True,["simyear","jday","minTemp","maxTemp","precip","rad"])

In [ ]:
gendf.head()

In [ ]:
from aquacropgym.envs import CropEnv, nebraska_maize_config

In [ ]:
envconfig=nebraska_maize_config.copy()
envconfig['gendf']=gendf
envconfig['action_set']='binary'
# envconfig['observation_set']='forecast'
envconfig['days_to_irr']=1



In [ ]:
env=CropEnv(envconfig)
# env.get_mean_std(1000)

In [ ]:
env.mean

In [ ]:
env.std

In [ ]:
import ray

In [ ]:
ray.shutdown()

In [ ]:
ray.init(num_cpus=8,num_gpus=1)

In [ ]:
# from ray.rllib.agents.ppo import ppo

# config = ppo.DEFAULT_CONFIG.copy()

# config['num_workers'] = 8
# config['num_gpus'] = 0
# # config['num_envs_per_worker'] = 8
# # config['num_sgd_iter'] = 30
# # config['lr'] = 3e-5
# # config['sgd_minibatch_size'] = 1028
# config['batch_mode'] = 'complete_episodes'
# config['rollout_fragment_length'] = 500
# config['train_batch_size'] = 4_000
# config['model']['fcnet_hiddens'] = [256]*2
# config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed
# config['framework'] = 'torch'
# config['gamma'] = 0.99999
# config['env_config']=envconfig
# # config['model']['use_lstm'] = True
# # config['model']['lstm_cell_size'] = 64
# # config['model']['max_seq_len'] = 40

In [ ]:
# from ray.rllib.agents.dqn.apex import ApexTrainer,APEX_DEFAULT_CONFIG

# config=APEX_DEFAULT_CONFIG.copy()
# config['gamma'] = 0.99999
# config['framework'] = 'torch'
# config['num_workers'] = 8
# config['num_gpus'] = 0
# config['hiddens'] = [256]*2
# config['buffer_size'] = 400_000
# config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed
# config['env_config']=envconfig


In [ ]:
from ray.rllib.agents.impala import impala

config = impala.DEFAULT_CONFIG.copy()

config['num_workers'] = 8
config['num_gpus'] = 0
# config['num_envs_per_worker'] = 8
# config['num_sgd_iter'] = 30
# config['lr'] = 3e-5
# config['sgd_minibatch_size'] = 1028
# config['batch_mode'] = 'complete_episodes'
config['rollout_fragment_length'] = 500
config['train_batch_size'] = 4_000
config['model']['fcnet_hiddens'] = [256]*2
config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed
config['framework'] = 'torch'
config['gamma'] = 0.99999
config['env_config']=envconfig
# config['model']['use_lstm'] = True
# config['model']['lstm_cell_size'] = 64
# config['model']['max_seq_len'] = 40

In [ ]:
# agent = ppo.PPOTrainer(config, CropEnv)
agent = impala.ImpalaTrainer(config, CropEnv)
# agent = ApexTrainer(config, CropEnv)

In [ ]:
from aquacropgym.utils import evaluate_agent
import copy

In [ ]:
# test_env_config=copy.deepcopy(envconfig)
# test_env_config['evaluation_run']=True
# config['env_config']['days_to_irr']=1

# train_rew, test_rew = evaluate_agent(agent,CropEnv,test_env_config)

# print(f'Train:{round(train_rew,3)}')
# print(f'Test:{round(test_rew,3)}')

In [ ]:
# testconfig=copy.deepcopy(config)
# # testconfig['env_config']['days_to_irr']=3

# agent = ppo.PPOTrainer(testconfig, CropEnv)
# agent = ppo.PPOTrainer(config, CropEnv)

# checkpoint_path="/home/mbyx3tk2/ray_results/PPO_CropEnv_2022-02-18_14-57-34s8iz8dhe/checkpoint_000270/checkpoint-270"

# # checkpoint_path
# agent.restore(checkpoint_path)




In [ ]:
proftrain=[]
proftest=[]
timesteps=[]

for i in range(1,500001):
    result = agent.train()

    ts = result['timesteps_total']

    #print(pretty_print(result))

    if i%10==0:
        checkpoint_path = agent.save()
        print(checkpoint_path)
        # trained_config = config.copy()
        # test_agent =ppo PPOTrainer(trained_config, CropEnv)
        # test_agent.restore(checkpoint_path)
        result_df = pd.DataFrame([timesteps,proftrain,proftest]).T
        result_df.to_csv('ppo70_day7_smt4_dayshift_noforecast_neweto_endreward.csv')
        plt.plot(timesteps,proftrain)
        plt.plot(timesteps,proftest)
        plt.show()
        if timesteps[-1]>=40_000_000:
            break

    if i%3==0:

        test_env_config=copy.deepcopy(envconfig)
        test_env_config['evaluation_run']=True
        train_rew, test_rew = evaluate_agent(agent,CropEnv,test_env_config)
        
        proftrain.append(train_rew)
        proftest.append(test_rew)
        timesteps.append(ts)

        print(f'Train:{round(train_rew,3)}')
        print(f'Test:{round(test_rew,3)}')


In [ ]:
# from ray.rllib.agents.impala import ImpalaTrainer, DEFAULT_CONFIG
# from ray.rllib.agents.sac import SACTrainer, DEFAULT_CONFIG
# from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG
# from ray.rllib.agents.dqn.apex import ApexTrainer, APEX_DEFAULT_CONFIG


In [ ]:
# config=DEFAULT_CONFIG.copy()
# config['gamma'] = 0.99999
# config['framework'] = 'torch'
# config['num_workers'] = 8
# config['num_gpus'] = 0
# config['model']['fcnet_hiddens'] = [128]*4
# config['rollout_fragment_length'] = 160
# config['train_batch_size'] = 4000
# config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed



In [ ]:
# config = DEFAULT_CONFIG.copy()
# config['num_workers'] = 16
# config['num_gpus'] = 1
# # config['num_envs_per_worker'] = 8
# # config['num_sgd_iter'] = 30
# # config['lr'] = 3e-5
# # config['sgd_minibatch_size'] = 1028
# # config['train_batch_size'] = 1028
# # config['timesteps_per_iteration'] = 4000
# # config['Q_model']['fcnet_hiddens'] = [128]*4
# # config['policy_model']['fcnet_hiddens'] = [128]*4
# config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed
# config['framework'] = 'torch'
# config['gamma'] = 0.99999
# # config['initial_alpha'] = 0.2
# config['optimization']['actor_learning_rate'] = 1e-2
# config['optimization']['critic_learning_rate'] = 1e-2
# config['optimization']['entropy_learning_rate'] = 1e-2

# # config['model']['use_lstm'] = True
# # config['model']['lstm_cell_size'] = 64
# # config['model']['max_seq_len'] = 40

In [ ]:
# config = DEFAULT_CONFIG.copy()
# config['num_workers'] = 8
# config['num_gpus'] = 0
# # config['num_envs_per_worker'] = 8
# # config['num_sgd_iter'] = 30
# # config['lr'] = 3e-5
# # config['sgd_minibatch_size'] = 1028
# config['train_batch_size'] = 4000
# config['model']['fcnet_hiddens'] = [128]*4
# config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed
# config['framework'] = 'torch'
# config['gamma'] = 0.99999
# config['env_config']={"evaluation_run":False}
# config['model']['use_lstm'] = True
# config['model']['lstm_cell_size'] = 64
# config['model']['max_seq_len'] = 40

In [ ]:
# config = DEFAULT_CONFIG.copy()
# config['num_workers'] = 8
# # config['num_envs_per_worker'] = 8
# # config['num_sgd_iter'] = 30
# config['lr'] = 3e-5
# # config['sgd_minibatch_size'] = 1028
# config['train_batch_size'] = 1028
# config['Q_model']['fcnet_hiddens'] = [128]*4
# config['policy_model']['fcnet_hiddens'] = [128]*4
# config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed
# config['framework'] = 'torch'
# config['gamma'] = 0.99999
# config['target_network_update_freq'] = 1

# # config['model']['use_lstm'] = True
# # config['model']['lstm_cell_size'] = 64
# # config['model']['max_seq_len'] = 17

In [ ]:
# agent = PPOTrainer(config, CropEnv)
# agent = SACTrainer(config, CropEnv)
# agent = ImpalaTrainer(config, CropEnv)
# agent = ApexTrainer(config, CropEnv)


# checkpoint_path
# agent.restore(checkpoint_path)